# Assignment 1: Hello World
## Edoardo Vassallo - S4965918

# 1 - Load and display meshes
Code in this section should run out-of-the-box and show you the three meshes in input. 

In [14]:
import numpy as np
import igl
import meshplot

In [15]:
bunny_v, bunny_f    = igl.read_triangle_mesh("data/bunny.off")
cube_v, cube_f      = igl.read_triangle_mesh("data/cube.obj")
sphere_v, sphere_f  = igl.read_triangle_mesh("data/sphere.obj")

In [16]:
meshplot.plot(bunny_v, bunny_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [17]:
meshplot.plot(cube_v, cube_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [18]:
meshplot.plot(sphere_v, sphere_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# 2. - Neighbourhood Computations

## 2.1 - Vertex-to-Face Relations

In [19]:
def VF_list(v, f):
    # Compute the following:
    # VF: vertex to face adjacency list
    # NI: vertex to face adjacency list offsets
    vf, ni = igl.vertex_triangle_adjacency(f, v.shape[0])
    # Create an actual list from the previous step
    adjacency_list = []
    for i in range(v.shape[0]):
        adjacency_list.append(vf[ni[i]:ni[i+1]])
    return adjacency_list

cube_adjacency_list = VF_list(cube_v, cube_f)

# Print and text dump the adjacency list
with open("./output/text_dumps/cube_VF_list.txt", "w") as file:
    for v_idx, face_indices in enumerate(cube_adjacency_list):
        print(f"Vertex {v_idx}: Faces {list(face_indices)}")
        file.write(f"Vertex {v_idx}: Faces {list(face_indices)}\n")

Vertex 0: Faces [0, 6, 7, 10, 11]
Vertex 1: Faces [0, 1, 7, 8]
Vertex 2: Faces [0, 1, 2, 11]
Vertex 3: Faces [1, 2, 3, 8, 9]
Vertex 4: Faces [2, 3, 4, 10, 11]
Vertex 5: Faces [3, 4, 5, 9]
Vertex 6: Faces [4, 5, 6, 10]
Vertex 7: Faces [5, 6, 7, 8, 9]


## 2.2 - Vertex-to-Vertex Relations

In [20]:
# Compute VV: vertex-vertex adjacency list
VV = igl.adjacency_list(cube_f)

# Print and text dump the adjacency list
with open("./output/text_dumps/cube_VV_list.txt", "w") as file:
    for v_idx, neighbors in enumerate(VV):
        print(f"Vertex {v_idx}: Neighbors {list(neighbors)}")
        file.write(f"Vertex {v_idx}: Neighbors {list(neighbors)}\n")

Vertex 0: Neighbors [1, 2, 4, 6, 7]
Vertex 1: Neighbors [0, 2, 3, 7]
Vertex 2: Neighbors [0, 1, 3, 4]
Vertex 3: Neighbors [1, 2, 4, 5, 7]
Vertex 4: Neighbors [0, 2, 3, 5, 6]
Vertex 5: Neighbors [3, 4, 6, 7]
Vertex 6: Neighbors [0, 4, 5, 7]
Vertex 7: Neighbors [0, 1, 3, 5, 6]


## 2.3 - Visualizing the Neighborhood Relations

In [21]:
# Print cube contents to compare with previous output
print(cube_f)
print(cube_v)

[[0 1 2]
 [2 1 3]
 [2 3 4]
 [4 3 5]
 [4 5 6]
 [6 5 7]
 [6 7 0]
 [0 7 1]
 [1 7 3]
 [3 7 5]
 [6 0 4]
 [4 0 2]]
[[-0.5 -0.5  0.5]
 [ 0.5 -0.5  0.5]
 [-0.5  0.5  0.5]
 [ 0.5  0.5  0.5]
 [-0.5  0.5 -0.5]
 [ 0.5  0.5 -0.5]
 [-0.5 -0.5 -0.5]
 [ 0.5 -0.5 -0.5]]


# 3 - Shading

Meshplot requires per vertex normals. 
Smooth (per-vertex) shading is easy and can be done directly from the input mesh. 
For flat shading and per-corner shading, we need to "explode" the mesh, as if it were a triangle soup. 
Per-face and per-corner normals are computed by igl from the input mesh, but must be adapted later to apply them to the exploded mesh. 

In [22]:
# Explode the mesh
def explode_mesh(V, F):
    # get list of duplicated vertexes in faces order
    V_exploded = V[F].reshape(-1, 3)
    # create list of faces with indexes of exploded vertexes
    F_exploded = np.arange(len(V_exploded)).reshape(-1, 3)
    return V_exploded, F_exploded

# Explode the mesh for visualization of per-face and per-corner normals
bunny_exp_v, bunny_exp_f    = explode_mesh(bunny_v, bunny_f)
sphere_exp_v, sphere_exp_f  = explode_mesh(sphere_v, sphere_f)
cube_exp_v, cube_exp_f      = explode_mesh(cube_v, cube_f)


### 3.1 - Flat Shading

In [23]:
# Compute per-face normals
n_faces_bunny   = igl.per_face_normals(bunny_v, bunny_f, np.array([0.0, 0.0, 1.0]))
n_faces_cube    = igl.per_face_normals(cube_v, cube_f, np.array([0.0, 0.0, 1.0]))
n_faces_sphere  = igl.per_face_normals(sphere_v, sphere_f, np.array([0.0, 0.0, 1.0]))

# Display flat shading
meshplot.plot(bunny_exp_v, bunny_exp_f,     n = n_faces_bunny)
meshplot.plot(cube_exp_v, cube_exp_f,       n = n_faces_cube)
meshplot.plot(sphere_exp_v, sphere_exp_f,   n = n_faces_sphere)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### 3.2 - Per-vertex Shading

In [24]:
# Compute per-vertex normals
n_vertex_bunny  = igl.per_vertex_normals(bunny_v, bunny_f)
n_vertex_cube   = igl.per_vertex_normals(cube_v, cube_f)
n_vertex_sphere = igl.per_vertex_normals(sphere_v, sphere_f)

# Display smooth shading
meshplot.plot(bunny_v, bunny_f, n=n_vertex_bunny, shading={"flat": False})
meshplot.plot(cube_v, cube_f, n=n_vertex_cube, shading={"flat": False})
meshplot.plot(sphere_v, sphere_f, n=n_vertex_sphere, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### 3.3 - Per-corner Shading

In [25]:
# Compute per-corner normals for per-corner shading
n_corner_bunny  = igl.per_corner_normals(bunny_v, bunny_f, 45)
n_corner_cube   = igl.per_corner_normals(cube_v, cube_f, 45)
n_corner_sphere = igl.per_corner_normals(sphere_v, sphere_f, 45)

# Display per-corner shading
meshplot.plot(bunny_exp_v, bunny_exp_f, n=n_corner_bunny, shading={"flat": False})
meshplot.plot(cube_exp_v, cube_exp_f, n=n_corner_cube, shading={"flat": False})
meshplot.plot(sphere_exp_v, sphere_exp_f, n=n_corner_sphere, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# 4. - A simple subdivision scheme

In [26]:
def subdivide_sq3(V, F):
    # TBD: add iterative refinement support

    old_v = len(V) 
    V_N = V
    F_N = []

    # Compute new vertices
    for face in F:
        vertices = V[face]
        new_vertex = np.mean(vertices, axis=0)
        V_N = np.vstack([V_N, new_vertex])

    # Compute new faces
    F_N1, F_N2, F_N3= [], [], []
    for i, face in enumerate(F):
        v1, v2, v3 = face
        v4 = old_v + i
        F_N1.append([v4, v2, v3])
        F_N2.append([v1, v4, v3])
        F_N3.append([v1, v2, v4])
    F_N = np.vstack([F_N1, F_N2, F_N3])

    # Update old vertices
    neighbors = igl.adjacency_list(F)
    for i in range(old_v):
        n = len(neighbors[i])
        a_n = (4 - 2 * np.cos((2*np.pi)/n))/9
        V_N[i] = (1 - a_n) * V[i] + (a_n/n) * np.sum(V[neighbors[i]], axis=0)

    # Using the second suggested approach:
    # Compute edges
    edges = igl.edges(F_N)
    
    # Flip each old edge if it is not on the boundary
    for edge in edges:
        v1, v2 = edge
        if v1 < old_v and v2 < old_v:
            # Find the faces that contain the edge
            edge_set = set(edge) # convert to set to use subset search
            faces = [idx for idx, face in enumerate(F_N) if edge_set.issubset(face)]
            # If the edge is not on the boundary
            if len(faces) > 1:
                # flip the edge
                v3 = np.setdiff1d(F_N[faces[0]], [v1, v2])[0]
                v4 = np.setdiff1d(F_N[faces[1]], [v1, v2])[0]
                F_N[faces[0]] = [v1, v3, v4]
                F_N[faces[1]] = [v2, v3, v4]

    return np.array(V_N), np.array(F_N)

# Subdivide multiple times
def subdivide_sq3_iterative(V, F, n_iter):
    results = []
    V_N, F_N = V, F
    for i in range(n_iter):
        V_N, F_N = subdivide_sq3(V_N, F_N)
        meshplot.plot(V_N, F_N, shading={"wireframe": True})
        results.append([V_N, F_N])
    return results

In [27]:
# Subdivide bunny, cube, and sphere
sub_bunny_v, sub_bunny_f = subdivide_sq3(bunny_v, bunny_f)
sub_cube_v, sub_cube_f = subdivide_sq3(cube_v, cube_f)
sub_sphere_v, sub_sphere_f = subdivide_sq3(sphere_v, sphere_f)

# Display the results
meshplot.plot(sub_bunny_v, sub_bunny_f, shading={"wireframe": True})
meshplot.plot(sub_cube_v, sub_cube_f, shading={"wireframe": True})
meshplot.plot(sub_sphere_v, sub_sphere_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016826…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(2.9802322…

In [28]:
# Example of iterative subdivision
meshplot.plot(cube_v, cube_f, shading={"wireframe": True})
iterations = subdivide_sq3_iterative(cube_v, cube_f, 7)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…